In [61]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

import re
import random
import time
from tqdm import tqdm
import importlib
import utils.amazon_scrapper_for_product_recommendation
import numpy as np
import pandas as pd
import json

importlib.reload(utils.amazon_scrapper_for_product_recommendation)
from utils.amazon_scrapper_for_product_recommendation import amazon_scrapping

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
def get_proxies():
    ua=UserAgent()
    headers = {'User-Agent':ua.random}
    url='https://free-proxy-list.net/'

    r=requests.get(url,headers=headers)
    page = BeautifulSoup(r.text, 'html.parser')

    proxies=[]

    for proxy in page.find_all('tr'):
        i=ip=port=0

        for data in proxy.find_all('td'):
            if i==0:
                ip=data.get_text()
            if i==1:
                port=data.get_text()
            i+=1
    
        if ip!=0 and port!=0 and '.' in ip:
            proxies+=[{'http':'http://'+ip+':'+port}]

    return proxies

In [21]:
ua = UserAgent()

def get_random_headers():
        
    headers = {
        'authority': 'www.amazon.com',
        'pragma': 'no-cache',
        'cache-control': 'no-cache',
        'dnt': '1',
        'Connection': 'keep-alive',
        'upgrade-insecure-requests': '1',
        'user-agent': ua.random,
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'none',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-dest': 'document',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    }
    
    return headers

In [22]:
def search(phrase, headers, proxies_list):
    phrase = re.sub(r'[^a-zA-Z0-9]',' ',phrase)
    phrase = re.sub(' ','+',phrase)
        
    url_amazon = 'https://www.amazon.com/s?k={}&ref=nb_sb_noss_2'.format(phrase)

    proxies = random.choice(proxies_list)
    
    try:
        source_code = requests.get(url_amazon, headers=headers, proxies=proxies, timeout=10)
        # If response status code is 503, wait for a while and retry
        if source_code.status_code == 503:
            print("503 Error. Retrying after 5 seconds...")
            time.sleep(5)
            return search(phrase, headers, proxies_list)
        source_code.raise_for_status()  # Raise an error for any other HTTP status codes
    except Exception as e:
        print("Error occurred:", e)
        return None
    
    return source_code, url_amazon

proxies_list = get_proxies()
headers = get_random_headers()

response, url_amazon = search("Laptops with 8GB RAM", headers, proxies_list)

if response:
    print(response)

<Response [200]>


In [23]:
print(response.url)

https://www.amazon.com/s?k=Laptops+with+8GB+RAM&ref=nb_sb_noss_2


In [24]:
plain_text = response.text
soup = BeautifulSoup(plain_text, "html.parser")

In [110]:
def product_data_for_recommendation(product_data, similarity_score_column_required):
    product_data_overview = product_data['product_overview']
    product_data_overview_text = ". ".join([key + " is " + value 
                                            for key,value in product_data_overview.items()])
    if type(product_data['description']) == list:
        product_description_text = ". ".join(product_data['description'])
    else:
        product_description_text = product_data['description']
        
    product_data_title = product_data['Title']
    product_data_url = product_data['url']
    
    final_product_text_for_similarity = product_data_overview_text + ". " + product_description_text 
    
    if similarity_score_column_required:
        final_product_data = {'product_name' : product_data_title.strip(),
                              'product_overview': final_product_text_for_similarity.strip(),
                              'product_link': product_data_url,
                              'product_image_link': image_link['src'],
                              'similarity_score': 0}
    else:
        final_product_data = {'product_name' : product_data_title.strip(),
                              'product_overview': final_product_text_for_similarity.strip(),
                              'product_link': product_data_url,
                              'product_image_link': image_link['src']}
    
    return final_product_data

In [111]:
all_links = []

soup = BeautifulSoup(plain_text, "html.parser")

all_product_links = soup.find_all('a',
                    {"class":"a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"})  
all_image_links = soup.find_all('img', {"class": "s-image"})

# print("Number of products", len(all_product_links), len(all_image_links))

similar_products_df = pd.DataFrame(columns=['product_name', 'product_overview',
                                            'product_link', 'product_image_link',
                                            'similarity_score'])
counter = 0
counter_limit = 10

for link, image_link in tqdm(zip(all_product_links, all_image_links)): 
    product_link = "https://www.amazon.com" + link['href']
    
    try:
        proxies = random.choice(proxies_list)
        headers = get_random_headers()
        product_image_url = image_link['src']    
        
        if not 'www.amazon.comhttps' in product_link:
            product_data = amazon_scrapping(product_link, proxies, headers)
            
            product_data['image_link'] = image_link['src']
            
            final_product_data = product_data_for_recommendation(product_data,
                                                                 similarity_score_column_required=True)
            
            if len(final_product_data['product_overview']) > 1:
                
                similar_products_df = pd.concat([ similar_products_df, pd.DataFrame([final_product_data])], axis=0)
            
                counter += 1
                
                if counter >= counter_limit:
                    break
                
                
    except ConnectionError:
        break

22it [02:48,  7.65s/it]


In [112]:
similar_products_df.reset_index(inplace=True, drop=True)
similar_products_df

,product_name,product_overview,product_link,product_image_link,similarity_score
0,VGKE [Windows 11 Home] B15 Windows 11 Laptop w...,. Intel Strong Quad-Core: High-performance Int...,https://www.amazon.com/VGKE-B15-Fingerprint-Pr...,https://m.media-amazon.com/images/I/61MFntlabu...,0
1,"Lenovo Ideapad Laptop (2024) Business Student,...",Brand is Lenovo. Model Name is Newest Lenovo 1...,https://www.amazon.com/Lenovo-Laptop-14-HD-Mar...,https://m.media-amazon.com/images/I/81UGIl1NGR...,0
2,"Oemgenuine Lenovo ThinkPad E16, WUXGA 16"" IPS,...",Brand is Oemgenuine. Model Name is ThinkPad E1...,https://www.amazon.com/Oemgenuine-ThinkPad-E16...,https://m.media-amazon.com/images/I/71vLLnhiMJ...,0
3,"URAO 15.6'' Laptop 8GB DDR4 256GB SSD, Quad-Co...",. Efficient Performance URAO laptop is equippe...,https://www.amazon.com/URAO-Quad-Core-Processo...,https://m.media-amazon.com/images/I/11++B3A2NE...,0
4,"OTVOC Laptop 15.6 inch Windows 11, VocBook 15,...",. Intel 11th-Gen Quad core processor: With Cel...,https://www.amazon.com/OTVOC-VocBook-15-Expans...,https://m.media-amazon.com/images/I/111mHoVK0k...,0
5,2021 Newest ASUS VivoBook Ultra Thin and Light...,Brand is ASUS. Model Name is ASUS VivoBook. Sc...,https://www.amazon.com/ASUS-Touchscreen-Quad-C...,https://m.media-amazon.com/images/I/611d8eRL+4...,0
6,"Chicbuy 15.6'' Laptop Computer, 8GB RAM Laptop...",Brand is Chicbuy. Model Name is LT1504. Screen...,https://www.amazon.com/Chicbuy-Computer-Proces...,https://m.media-amazon.com/images/I/71Ezvxd+ui...,0
7,"INHONLAP Laptops, 15.6"" FHD IPS Display Laptop...",Brand is INHONLAP. Model Name is N15. Screen S...,https://www.amazon.com/INHONLAP-Computers-Quad...,https://m.media-amazon.com/images/I/71TwgF7rw8...,0
8,"URAO 15.6'' Laptop 8GB DDR4 256GB SSD, Quad-Co...",Brand is URAO. Model Name is LT1504. Screen Si...,https://www.amazon.com/URAO-Quad-Core-Processo...,https://m.media-amazon.com/images/I/81O3TvrFnU...,0
9,2021 Newest ASUS VivoBook Ultra Thin and Light...,Brand is ASUS. Model Name is ASUS VivoBook. Sc...,https://www.amazon.com/ASUS-Touchscreen-Quad-C...,https://m.media-amazon.com/images/I/71K5Hmg7jM...,0


In [87]:
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')  #'msmarco-distilbert-base-v4'

In [89]:
current_product_data = similar_products_df.iloc[0]['product_overview']
current_product_data

". Intel Strong Quad-Core: High-performance Intel J4125 processor, running fast, fast processing, multiple programs free switch. Processor main frequency up to 2.7 GHz.. FINGERPRINT READER: B15 laptop comes with convenient fingerprint reader. Strengthen the protection of personal privacy and information security.. FULL-SIZE BACKLIT KEYBOARDB15 laptop has a full-size keyboard with backlight, improves the convenience of input, the independent numeric keypad makes input even quick and convenient. Press Fn+F5 at the same time to turn on the backlit keyboard.. 12GB DDR4 and 256GB SSD: The laptop runs with 12 GB DDR4 dual-channel RAM, which can greatly improve the system response speed. 256GB SSD much faster than traditional mechanical hard disk, provides fast reading and writing experience.. FULL HD SCREEN: B15 has a 15.6 inch 1920*1080 Full HD IPS display with 5.9mm narrow bezels, you'll enjoy more screen real estate and less clutter."

In [121]:
def similarity_scores(similar_products_df, current_product_data):
    
    for i in range(1, len(similar_products_df)):
        text_1 = model.encode(similar_products_df.iloc[i]['product_overview'])
        base_text = model.encode(current_product_data)
        score = cosine_similarity([text_1], [base_text])[0][0]
        similar_products_df.loc[i, 'similarity_score'] = score
    
    sorted_similar_products_df = similar_products_df.sort_values('similarity_score',
                                                                 ascending=False).reset_index(drop=True)
    
    return sorted_similar_products_df

In [122]:
sorted_similar_products_df = similarity_scores(similar_products_df, current_product_data)
sorted_similar_products_df

,product_name,product_overview,product_link,product_image_link,similarity_score
0,"OTVOC Laptop 15.6 inch Windows 11, VocBook 15,...",. Intel 11th-Gen Quad core processor: With Cel...,https://www.amazon.com/OTVOC-VocBook-15-Expans...,https://m.media-amazon.com/images/I/111mHoVK0k...,0.66444
1,"Chicbuy 15.6'' Laptop Computer, 8GB RAM Laptop...",Brand is Chicbuy. Model Name is LT1504. Screen...,https://www.amazon.com/Chicbuy-Computer-Proces...,https://m.media-amazon.com/images/I/71Ezvxd+ui...,0.596675
2,"Lenovo Ideapad Laptop (2024) Business Student,...",Brand is Lenovo. Model Name is Newest Lenovo 1...,https://www.amazon.com/Lenovo-Laptop-14-HD-Mar...,https://m.media-amazon.com/images/I/81UGIl1NGR...,0.562485
3,2021 Newest ASUS VivoBook Ultra Thin and Light...,Brand is ASUS. Model Name is ASUS VivoBook. Sc...,https://www.amazon.com/ASUS-Touchscreen-Quad-C...,https://m.media-amazon.com/images/I/611d8eRL+4...,0.533611
4,2021 Newest ASUS VivoBook Ultra Thin and Light...,Brand is ASUS. Model Name is ASUS VivoBook. Sc...,https://www.amazon.com/ASUS-Touchscreen-Quad-C...,https://m.media-amazon.com/images/I/71K5Hmg7jM...,0.533611
5,"Oemgenuine Lenovo ThinkPad E16, WUXGA 16"" IPS,...",Brand is Oemgenuine. Model Name is ThinkPad E1...,https://www.amazon.com/Oemgenuine-ThinkPad-E16...,https://m.media-amazon.com/images/I/71vLLnhiMJ...,0.49122
6,"INHONLAP Laptops, 15.6"" FHD IPS Display Laptop...",Brand is INHONLAP. Model Name is N15. Screen S...,https://www.amazon.com/INHONLAP-Computers-Quad...,https://m.media-amazon.com/images/I/71TwgF7rw8...,0.4415
7,"URAO 15.6'' Laptop 8GB DDR4 256GB SSD, Quad-Co...",. Efficient Performance URAO laptop is equippe...,https://www.amazon.com/URAO-Quad-Core-Processo...,https://m.media-amazon.com/images/I/11++B3A2NE...,0.399952
8,"URAO 15.6'' Laptop 8GB DDR4 256GB SSD, Quad-Co...",Brand is URAO. Model Name is LT1504. Screen Si...,https://www.amazon.com/URAO-Quad-Core-Processo...,https://m.media-amazon.com/images/I/81O3TvrFnU...,0.375959
9,VGKE [Windows 11 Home] B15 Windows 11 Laptop w...,. Intel Strong Quad-Core: High-performance Int...,https://www.amazon.com/VGKE-B15-Fingerprint-Pr...,https://m.media-amazon.com/images/I/61MFntlabu...,0
